## Pure Elements

In [5]:
from ase.lattice.cubic import BodyCenteredCubic
from ase.visualize import view
atoms = BodyCenteredCubic('W')
view(atoms.repeat((2,2,2)), viewer = 'x3d')

In [6]:
from pymatgen.io.ase import AseAtomsAdaptor
structure = AseAtomsAdaptor.get_structure(atoms)
structure

Structure Summary
Lattice
    abc : 3.16 3.16 3.16
 angles : 90.0 90.0 90.0
 volume : 31.554496000000007
      A : np.float64(3.16) np.float64(0.0) np.float64(0.0)
      B : np.float64(0.0) np.float64(3.16) np.float64(0.0)
      C : np.float64(0.0) np.float64(0.0) np.float64(3.16)
    pbc : True True True
PeriodicSite: W (0.0, 0.0, 0.0) [0.0, 0.0, 0.0]
PeriodicSite: W (1.58, 1.58, 1.58) [0.5, 0.5, 0.5]

In [8]:
from pymatgen.io.vasp.sets import MPRelaxSet
MPRelaxSet(structure).write_input("/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/pure/w_mprelaxset",
                                  potcar_spec={"W": "W_sv"})

/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/elastic_mlps/fairchem/lib/python3.11/site-packages/pymatgen/io/vasp/sets.py:483: BadInputSetWarning: Relaxation of likely metal with ISMEAR < 0 (-5). See VASP recommendations on ISMEAR for metals (https://www.vasp.at/wiki/index.php/ISMEAR).
  incar=self.incar,


In [ ]:
import os
from ase.lattice.cubic import BodyCenteredCubic
from ase.io import write
from pymatgen.core import Structure
from pymatgen.io.vasp.inputs import Kpoints
import shutil

# --- Elements to process ---
elements = ["W", "Ta", "Nb"]

# Base output directory
base_outdir = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/pure"

# INCAR settings (same for all)
incar_settings = {
    "GGA": "PE",
    "ALGO": "Normal",
    "EDIFF": 1e-07,
    "EDIFFG": -0.001,
    "ENCUT": 520.0,
    "IBRION": 6,
    "ISIF": 3,
    "ISMEAR": 1,
    "ISPIN": 1,
    "LASPH": True,
    "LMAXMIX": 6,
    "LORBIT": 11,
    "LREAL": False,
    "LWAVE": False,
    "NELM": 100,
    "NSW": 99,
    "PREC": "Accurate",
    "SIGMA": 0.2,
    "POTIM": 0.01,
}

# POTCAR library mapping
potcar_map = {
    "W": "POTCAR.W_sv",
    "Ta": "POTCAR.Ta_pv",
    "Nb": "POTCAR.Nb_sv",
}

potcar_library = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/POT_GGA_PAW_PBE_54"

# Loop over elements
for el in elements:
    # --- Build ASE atoms object ---
    atoms = BodyCenteredCubic(el)

    # --- Output directory ---
    outdir = os.path.join(base_outdir, el.lower())
    os.makedirs(outdir, exist_ok=True)

    # --- Write POSCAR ---
    poscar_path = os.path.join(outdir, "POSCAR")
    write(poscar_path, atoms, format="vasp")

    # --- Write INCAR ---
    incar_path = os.path.join(outdir, "INCAR")
    with open(incar_path, "w") as f:
        for key, val in incar_settings.items():
            f.write(f"{key} = {val}\n")

    # --- Write KPOINTS ---
    structure = Structure.from_file(poscar_path)
    kpoints_density = 7000
    kpoints = Kpoints.automatic_density(structure, kpoints_density)
    kpoints.write_file(os.path.join(outdir, "KPOINTS"))

    # --- Write POTCAR ---
    potcar_choice = potcar_map[el]
    potcar_src = os.path.join(potcar_library, potcar_choice)
    potcar_dest = os.path.join(outdir, "POTCAR")
    shutil.copy(potcar_src, potcar_dest)

    print(f"Generated VASP input files for {el} in {outdir}")

Generated VASP input files for W in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/non_mag/w
Generated VASP input files for Ta in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/non_mag/ta
Generated VASP input files for Nb in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/non_mag/nb


## Ordered Structures

In [1]:
import json
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor

# Load from JSON
with open("relaxed_structures_with_energies.json", "r") as f:
    data = json.load(f)

# Reconstruct formation_energies as a list of (ase.Atoms, formation_energy)
formation_energies = []
for entry in data:
    pmg_struct = Structure.from_dict(entry["structure"])
    atoms = AseAtomsAdaptor.get_atoms(pmg_struct)
    formation_energies.append((atoms, entry["formation_energy"]))

In [2]:
from collections import defaultdict
from tqdm import tqdm
import numpy as np

lowest_by_comp = defaultdict(lambda: (None, np.inf))

for atoms, E_form in tqdm(formation_energies):
    formula = atoms.get_chemical_formula()
    if E_form < lowest_by_comp[formula][1]:
        lowest_by_comp[formula] = (atoms, E_form)

# print results
for formula, (atoms, E_form) in lowest_by_comp.items():
    print(f"{formula}: lowest formation energy = {E_form:.4f} eV/atom")

  0%|          | 0/4764 [00:00<?, ?it/s]

100%|██████████| 4764/4764 [00:00<00:00, 60044.49it/s]

NbW9: lowest formation energy = -0.0282 eV/atom
NbTaW8: lowest formation energy = -0.0623 eV/atom
NbTa2W7: lowest formation energy = -0.0827 eV/atom
NbTa3W6: lowest formation energy = -0.0895 eV/atom
NbTa4W5: lowest formation energy = -0.0882 eV/atom
NbTa5W4: lowest formation energy = -0.0799 eV/atom
NbTa6W3: lowest formation energy = -0.0638 eV/atom
NbTa7W2: lowest formation energy = -0.0442 eV/atom
NbTa8W: lowest formation energy = -0.0237 eV/atom
NbTa9: lowest formation energy = -0.0008 eV/atom


## First, relax the structures

In [ ]:
import os
import shutil
from pymatgen.core import Structure
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Incar
from pymatgen.io.ase import AseAtomsAdaptor

# --- Base directories ---
base_outdir = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations"
potcar_library = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/POT_GGA_PAW_PBE_54"

# --- INCAR settings ---
incar_settings = {
    "ISYM": 0,
    "GGA": "PE",
    "ALGO": "Normal",
    "EDIFF": 1e-07,
    "EDIFFG": -0.001,
    "ENCUT": 520.0,
    "IBRION": 2,
    "ISIF": 3,
    "ISMEAR": 1,
    "ISPIN": 1,
    "LASPH": True,
    "LMAXMIX": 6,
    "LORBIT": 11,
    "LREAL": False,
    "LWAVE": False,
    "NELM": 100,
    "NSW": 200,
    "PREC": "Accurate",
    "SIGMA": 0.2,
    "NEDOS":9999,
}

# --- POTCAR mapping ---
potcar_map = {
    "W": "POTCAR.W_sv",
    "Ta": "POTCAR.Ta_pv",
    "Nb": "POTCAR.Nb_sv",
}

# Desired element order for all alloys
desired_order = ["Nb", "Ta", "W"]

# --- Loop over all alloys ---
for comp, atoms_list in lowest_by_comp.items():

    atoms = atoms_list[0]  # get ASE Atoms object

    # Convert ASE Atoms → pymatgen Structure
    structure = AseAtomsAdaptor.get_structure(atoms)

    # Reorder sites according to desired order
    sites_sorted = []
    for el in desired_order:
        sites_sorted.extend([s for s in structure.sites if s.specie.symbol == el])
    structure_sorted = Structure.from_sites(sites_sorted)

    # --- Target directory ---
    outdir = os.path.join(base_outdir, comp.lower())
    os.makedirs(outdir, exist_ok=True)

    # --- POSCAR ---
    poscar = Poscar(structure_sorted)
    poscar_path = os.path.join(outdir, "POSCAR")
    poscar.write_file(poscar_path)

    # --- INCAR ---
    incar = Incar(incar_settings)
    incar.write_file(os.path.join(outdir, "INCAR"))

    # --- KPOINTS ---
    kpoints_density = 7000
    kpoints = Kpoints.automatic_density(structure_sorted, kpoints_density)
    kpoints.write_file(os.path.join(outdir, "KPOINTS"))

    # --- POTCAR ---
    elements_in_structure = [el.symbol for el in structure_sorted.composition.elements]
    potcar_dest = os.path.join(outdir, "POTCAR")
    with open(potcar_dest, "wb") as wfd:
        for el in elements_in_structure:
            pc_file = os.path.join(potcar_library, potcar_map[el])
            with open(pc_file, "rb") as rfd:
                shutil.copyfileobj(rfd, wfd)

    print(f"Generated input files for {comp} in {outdir}")

Generated input files for NbW9 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations/nbw9
Generated input files for NbTaW8 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations/nbtaw8
Generated input files for NbTa2W7 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations/nbta2w7
Generated input files for NbTa3W6 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations/nbta3w6
Generated input files for NbTa4W5 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations/nbta4w5
Generated input files for NbTa5W4 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/relaxations/nbta5w4
Generated input files for NbTa6W3 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/

## Second, compute elastic moduli

In [ ]:
import os
import shutil
from pymatgen.core import Structure
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Incar
from pymatgen.io.ase import AseAtomsAdaptor

# --- Base directories ---
base_outdir = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ordered/elastic"
potcar_library = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/POT_GGA_PAW_PBE_54"

# --- INCAR settings ---
incar_settings = {
    "ISYM": 0,
    "GGA": "PE",
    "ALGO": "Normal",
    "EDIFF": 1e-07,
    "EDIFFG": -0.001,
    "ENCUT": 520.0,
    "IBRION": 6,
    "ISIF": 3,
    "ISMEAR": 1,
    "ISPIN": 1,
    "LASPH": True,
    "LMAXMIX": 6,
    "LORBIT": 11,
    "LREAL": False,
    "LWAVE": False,
    "NELM": 100,
    "NSW": 99,
    "PREC": "Accurate",
    "SIGMA": 0.2,
    "POTIM": 0.01,
    "NEDOS":9999,
}

# --- POTCAR mapping ---
potcar_map = {
    "W": "POTCAR.W_sv",
    "Ta": "POTCAR.Ta_pv",
    "Nb": "POTCAR.Nb_sv",
}

# Desired element order for all alloys
desired_order = ["Nb", "Ta", "W"]

# --- Loop over all alloys ---
for comp, atoms_list in lowest_by_comp.items():

    atoms = atoms_list[0]  # get ASE Atoms object

    # Convert ASE Atoms → pymatgen Structure
    structure = AseAtomsAdaptor.get_structure(atoms)

    # Reorder sites according to desired order
    sites_sorted = []
    for el in desired_order:
        sites_sorted.extend([s for s in structure.sites if s.specie.symbol == el])
    structure_sorted = Structure.from_sites(sites_sorted)

    # --- Target directory ---
    outdir = os.path.join(base_outdir, comp.lower())
    os.makedirs(outdir, exist_ok=True)

    # --- POSCAR ---
    # poscar = Poscar(structure_sorted)
    # poscar_path = os.path.join(outdir, "POSCAR")
    # poscar.write_file(poscar_path)

    # --- INCAR ---
    incar = Incar(incar_settings)
    incar.write_file(os.path.join(outdir, "INCAR"))

    # --- KPOINTS ---
    kpoints_density = 7000
    kpoints = Kpoints.automatic_density(structure_sorted, kpoints_density)
    kpoints.write_file(os.path.join(outdir, "KPOINTS"))

    # --- POTCAR ---
    elements_in_structure = [el.symbol for el in structure_sorted.composition.elements]
    potcar_dest = os.path.join(outdir, "POTCAR")
    with open(potcar_dest, "wb") as wfd:
        for el in elements_in_structure:
            pc_file = os.path.join(potcar_library, potcar_map[el])
            with open(pc_file, "rb") as rfd:
                shutil.copyfileobj(rfd, wfd)

    print(f"Generated input files for {comp} in {outdir}")

Generated input files for NbW9 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/nbw9
Generated input files for NbTaW8 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/nbtaw8
Generated input files for NbTa2W7 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/nbta2w7
Generated input files for NbTa3W6 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/nbta3w6
Generated input files for NbTa4W5 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/nbta4w5
Generated input files for NbTa5W4 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/nbta5w4
Generated input files for NbTa6W3 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/ordered/n

## SQS structures

In [17]:
import json
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor

# Load the JSON file
with open("w_ta_nb_sqs_structures.json") as f:
    sqs_dicts = json.load(f)

adaptor = AseAtomsAdaptor()
sqs_structures = {}

for w_fraction, struct_dict in sqs_dicts.items():
    # Convert dict -> pymatgen Structure
    sqs_structures[float(w_fraction)] = Structure.from_dict(struct_dict)

In [ ]:
from pymatgen.io.vasp.inputs import Poscar, Kpoints
structure = sqs_structures[0.2]
sites_sorted = []
desired_order = ["Nb", "Ta", "W"]
for el in desired_order:
    sites_sorted.extend([s for s in structure.sites if s.specie.symbol == el])
structure_sorted = Structure.from_sites(sites_sorted)

## Make sure to use ALGO=Normal

## First, relax the structures
### Could be done multiple times untill max forces< 10-2

In [ ]:
import os
import shutil
from pymatgen.core import Structure
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Incar
from pymatgen.io.ase import AseAtomsAdaptor

# --- Base directories ---
base_outdir = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations"
potcar_library = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/POT_GGA_PAW_PBE_54"

# --- INCAR settings ---
incar_settings = {
    "ISYM": 0,
    "GGA": "PE",
    "ALGO": "Normal",
    "EDIFF": 1e-07,
    "EDIFFG": -0.001,
    "ENCUT": 520.0,
    "IBRION": 2,
    "ISIF": 3,
    "ISMEAR": 1,
    "ISPIN": 1,
    "LASPH": True,
    "LMAXMIX": 6,
    "LORBIT": 11,
    "LREAL": False,
    "LWAVE": False,
    "NELM": 100,
    "NSW": 200,
    "PREC": "Accurate",
    "SIGMA": 0.2,
    "NEDOS":9999,
}

# --- POTCAR mapping ---
potcar_map = {
    "W": "POTCAR.W_sv",
    "Ta": "POTCAR.Ta_pv",
    "Nb": "POTCAR.Nb_sv",
}

# Desired element order for all alloys
desired_order = ["Nb", "Ta", "W"]

# --- Loop over all alloys ---
for comp, structure in sqs_structures.items():

    # Reorder sites according to desired order
    sites_sorted = []
    for el in desired_order:
        sites_sorted.extend([s for s in structure.sites if s.specie.symbol == el])
    structure_sorted = Structure.from_sites(sites_sorted)

    # --- Target directory ---
    outdir = os.path.join(base_outdir, structure.reduced_formula.lower())
    os.makedirs(outdir, exist_ok=True)

    # --- POSCAR ---
    poscar = Poscar(structure_sorted)
    poscar_path = os.path.join(outdir, "POSCAR")
    poscar.write_file(poscar_path)

    # --- INCAR ---
    incar = Incar(incar_settings)
    incar.write_file(os.path.join(outdir, "INCAR"))

    # --- KPOINTS ---
    kpoints_density = 7000
    kpoints = Kpoints.automatic_density(structure_sorted, kpoints_density)
    kpoints.write_file(os.path.join(outdir, "KPOINTS"))

    # --- POTCAR ---
    elements_in_structure = [el.symbol for el in structure_sorted.composition.elements]
    potcar_dest = os.path.join(outdir, "POTCAR")
    with open(potcar_dest, "wb") as wfd:
        for el in elements_in_structure:
            pc_file = os.path.join(potcar_library, potcar_map[el])
            with open(pc_file, "rb") as rfd:
                shutil.copyfileobj(rfd, wfd)

    print(f"Generated input files for {structure.reduced_formula} in {outdir}")

Generated input files for Ta8NbW in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta8nbw
Generated input files for Ta7NbW2 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta7nbw2
Generated input files for Ta6NbW3 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta6nbw3
Generated input files for Ta5NbW4 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta5nbw4
Generated input files for Ta4NbW5 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta4nbw5
Generated input files for Ta3NbW6 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta3nbw6
Generated input files for Ta2NbW7 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/relaxations/ta

## Second, compute elastic moduli

In [ ]:
import os
import shutil
from pymatgen.core import Structure
from pymatgen.io.vasp.inputs import Poscar, Kpoints, Incar
from pymatgen.io.ase import AseAtomsAdaptor

# --- Base directories ---
base_outdir = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/sqs/elastic"
potcar_library = "/home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/POT_GGA_PAW_PBE_54"

# --- INCAR settings ---
incar_settings = {
    "ISYM": 0,
    "GGA": "PE",
    "ALGO": "Normal",
    "EDIFF": 1e-07,
    "EDIFFG": -0.001,
    "ENCUT": 520.0,
    "IBRION": 6,
    "ISIF": 3,
    "ISMEAR": 1,
    "ISPIN": 1,
    "LASPH": True,
    "LMAXMIX": 6,
    "LORBIT": 11,
    "LREAL": False,
    "LWAVE": False,
    "NELM": 100,
    "NSW": 99,
    "PREC": "Accurate",
    "SIGMA": 0.2,
    "POTIM": 0.01,
    "NEDOS":9999,
}

# --- POTCAR mapping ---
potcar_map = {
    "W": "POTCAR.W_sv",
    "Ta": "POTCAR.Ta_pv",
    "Nb": "POTCAR.Nb_sv",
}

# Desired element order for all alloys
desired_order = ["Nb", "Ta", "W"]

# --- Loop over all alloys ---
for comp, structure in sqs_structures.items():

    # Reorder sites according to desired order
    sites_sorted = []
    for el in desired_order:
        sites_sorted.extend([s for s in structure.sites if s.specie.symbol == el])
    structure_sorted = Structure.from_sites(sites_sorted)

    # --- Target directory ---
    outdir = os.path.join(base_outdir, structure.reduced_formula.lower())
    os.makedirs(outdir, exist_ok=True)

    # # --- POSCAR ---
    # poscar = Poscar(structure_sorted)
    # poscar_path = os.path.join(outdir, "POSCAR")
    # poscar.write_file(poscar_path)

    # --- INCAR ---
    incar = Incar(incar_settings)
    incar.write_file(os.path.join(outdir, "INCAR"))

    # --- KPOINTS ---
    kpoints_density = 7000
    kpoints = Kpoints.automatic_density(structure_sorted, kpoints_density)
    kpoints.write_file(os.path.join(outdir, "KPOINTS"))

    # --- POTCAR ---
    elements_in_structure = [el.symbol for el in structure_sorted.composition.elements]
    potcar_dest = os.path.join(outdir, "POTCAR")
    with open(potcar_dest, "wb") as wfd:
        for el in elements_in_structure:
            pc_file = os.path.join(potcar_library, potcar_map[el])
            with open(pc_file, "rb") as rfd:
                shutil.copyfileobj(rfd, wfd)

    print(f"Generated input files for {structure.reduced_formula} in {outdir}")

Generated input files for Ta8NbW in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta8nbw
Generated input files for Ta7NbW2 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta7nbw2
Generated input files for Ta6NbW3 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta6nbw3
Generated input files for Ta5NbW4 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta5nbw4
Generated input files for Ta4NbW5 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta4nbw5
Generated input files for Ta3NbW6 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta3nbw6
Generated input files for Ta2NbW7 in /home/jovyan/shared-scratch-kabdelma-pvc/kabdelma/rare-earth/vasp_internal_elastic/ismear_1/sqs/ta2nbw7
Generated input